In [17]:
!pip install -r ..\salma_deployment\requirements.txt

In [19]:
import streamlit as st
import pickle
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime


In [59]:

# Load the models and transformers
with open('salma_log_reg.pkl', 'rb') as file:
    log_reg = pickle.load(file)

with open('salma_xgb_model.pkl', 'rb') as file:
    xgb_model = pickle.load(file)

with open('salma_vectorizer.pkl', 'rb') as file:
    vectorizer = pickle.load(file)

with open('salma_scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
    

In [65]:
def load_functions(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

functions_dict = load_functions('salma_functions.pkl')
print("Functions Loaded:", functions_dict)

clean_text_func = functions_dict['clean_text']
collect_headlines_func = functions_dict['collect_headlines']
fetch_spy_data_func = functions_dict['fetch_spy_data']


Functions Loaded: {'clean_text': <function clean_text at 0x000001D7D98D4860>, 'collect_headlines': <function collect_headlines at 0x000001D7EEEBBB00>, 'fetch_spy_data': <function fetch_spy_data at 0x000001D7EEEE05E0>}


In [71]:

# Streamlit app layout
st.title('Stock Movement Prediction')

# User inputs
# Convert to DataFrame
df_headlines = pd.DataFrame(collect_headlines_func())
# Apply clean_text function to the 'text' column
df_headlines['cleaned_text'] = df_headlines['text'].apply(clean_text_func)


[nltk_data] Downloading package punkt to nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stop

In [77]:

# Select the 'text' column
text_input = df_headlines['cleaned_text']


open_price_data = fetch_spy_data_func(datetime.datetime.now())


In [79]:
if st.button('Predict'):
    # Transform the input
    text_tfidf = vectorizer.transform(text_input)
     # Extract the 'Open' price from the fetched data
    open_price_input = open_price_data[0]['Open']
        
        # Transform the opening price, reshape to 2D array
    open_scaled = scaler.transform(np.array([[open_price_input]]))
    
    # Make predictions
    text_prediction = log_reg.predict(text_tfidf)
    open_prediction = xgb_model.predict(open_scaled)
    
    # Combine predictions
    combined_prediction = (text_prediction[0] + open_prediction[0]) / 2
    final_prediction = int(np.round(combined_prediction))
    
    st.write(f'Predicted Movement: {"Up" if final_prediction == 1 else "Down"}')

In [81]:
# Display the headlines
st.write("### News Headlines")
st.dataframe(df_headlines[['text']])

DeltaGenerator()

In [83]:
df_headlines[['text']]

,text
0,"Fed survey shows lows in employment, worries a..."
1,Goldman Sachs cuts odds of U.S. recession to 2...
2,Treasury yields waver ahead of Fed minutes and...
3,"Trump, Harris inflation arguments are question..."
4,"With Harris economic plan, U.S. family and chi..."
5,Big fast-casual restaurant chains are rapidly ...
6,How a week's worth of huge economic news will ...
7,Treasury yields fell as investors absorbed wee...
8,Here's the deflation breakdown for July 2024 —...
9,Consumer spending jumped in July as retail sal...


In [85]:
open_price_data

[{'Date': Timestamp('2024-08-19 00:00:00-0400', tz='America/New_York'),
  'Open': 554.72998046875}]

In [91]:
open_price_data[0]['Date']

Timestamp('2024-08-19 00:00:00-0400', tz='America/New_York')